In [1]:
%load_ext blackcellmagic

from datetime import datetime
from itertools import combinations, permutations
from numpy import genfromtxt
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import networkx as nx
import matplotlib.pyplot as plt


In [2]:
pc_data = genfromtxt("/home/sep/Desktop/pc-algorithms/data/pc-data.csv")


confidence_level = 0.02


def adjacent_count(adj_mat, i):
    return np.count_nonzero(adj_mat[i, :])


def loop1_check(adj_mat, lvl):
    for i in range(adj_mat.shape[1] - 1):
        for j in range(i + 1, adj_mat.shape[1]):
            if adjacent_count(adj_mat, i) - 1 >= lvl:
                return False

    return True


def partial_correlation(i, j, k_sub, data, test_type):
    if test_type is "recursive":
        if len(k_sub) == 0:
            return np.corrcoef(data[:, i], data[:, j])[0, 1]

        else:
            rho_ij_kh = partial_correlation(i, j, k_sub[1:], data, "recursive")
            rho_ih_kh = partial_correlation(i, k_sub[0], k_sub[1:], data, "recursive")
            rho_jh_kh = partial_correlation(j, k_sub[0], k_sub[1:], data, "recursive")

            return (rho_ij_kh - rho_ih_kh * rho_jh_kh) / np.sqrt(
                (1 - rho_ih_kh ** 2) * (1 - rho_jh_kh ** 2)
            )
        
    elif test_type is "regression":
        if len(k_sub) == 0:
            return np.corrcoef(data[:, i], data[:, j])[0, 1]
        
        else:
            i_residual = data[:, i] - LinearRegression().fit(data[:, k_sub], data[:, i]).predict(data[:, k_sub])
            
            j_residual = data[:, j] - LinearRegression().fit(data[:, k_sub], data[:, j]).predict(data[:, k_sub])
            
            return np.corrcoef(i_residual, j_residual)[0, 1]

def check_conditional_independence(conf_lvl, i, j, k_sub, data):
    rho_ij_k = partial_correlation(i, j, k_sub, data, "recursive")
#     print(k_sub)
    z_ij_k = np.log((1 + rho_ij_k) / (1 - rho_ij_k)) / 2
    if np.sqrt(data.shape[0] - len(k_sub) - 3) * np.abs(z_ij_k) <= norm.ppf(1 - conf_lvl / 2):
        return True
    else:
        return False
        


In [3]:
def pc_pop(data, conf_lvl, silent):
    adj_mat = np.ones((data.shape[1], data.shape[1])) - np.diag(
        [1 for i in range(data.shape[1])]
    )
    level = -1
    while True:
        if not silent:
            print(f"Level {level} completed. Current time: {datetime.now()}")
        level += 1
        for i, j in permutations(range(data.shape[1]), 2):
            if adj_mat[i, j] == 1 and adjacent_count(adj_mat, i) - 1 >= level:
                adj_i_j = list(np.nonzero(adj_mat[i, :])[0])
                adj_i_j.remove(j)
                for k in combinations(adj_i_j, level):
                    if check_conditional_independence(conf_lvl, i, j, k, data) is True:
                        adj_mat[i, j] = 0

                    if adj_mat[i, j] == 0:
                        break

        if loop1_check(adj_mat, level) is True:
            break

    return adj_mat


In [ ]:
estimated_skeleton = pc_pop(pc_data, confidence_level, 0)


Level -1 completed. Current time: 2018-12-15 11:38:43.976516
Level 0 completed. Current time: 2018-12-15 11:38:58.569686


In [ ]:
np.count_nonzero(estimated_skeleton)

In [ ]:
G_pop = nx.from_numpy_matrix(estimated_skeleton)
nx.draw(G_pop)
plt.show()

In [ ]:
def pc_stable(data, conf_lvl, silent):
    adj_mat = np.ones((data.shape[1], data.shape[1])) - np.diag(
        [1 for i in range(data.shape[1])]
    )
    level = -1
    removed_edges = []
    while True:
        if not silent:
            print(f"Level {level} completed. Current time: {datetime.now()}")
        level += 1
        for i, j in permutations(range(data.shape[1]), 2):
            if adj_mat[i, j] == 1 and adjacent_count(adj_mat, i) - 1 >= level:
                adj_i_j = list(np.nonzero(adj_mat[i, :])[0])
                adj_i_j.remove(j)
                for k in combinations(adj_i_j, level):
                    if check_conditional_independence(conf_lvl, i, j, k, data) is True:
                        removed_edges.append((i, j))

                    if adj_mat[i, j] == 0:
                        break

        if loop1_check(adj_mat, level) is True:
            break
            
    for i, j in removed_edges:
        adj_mat[i, j] = 0

    return adj_mat


In [ ]:
estimated_skeleton_stable = pc_stable(pc_data, confidence_level, 0)


In [ ]:
G_stable = nx.from_numpy_matrix(estimated_skeleton_stable)
nx.draw(G_stable)
plt.show()

In [ ]:
np.count_nonzero(estimated_skeleton_stable)

In [ ]:
def generate_random_graph(
    number_of_nodes, back_edge_probability, forward_edge_probability
):
    adj_mat = np.diag([1 for i in range(number_of_nodes)])
    for i, j in permutations(range(number_of_nodes), 2):
        if i < j:
            if np.random.binomial(1, forward_edge_probability):
                adj_mat[i, j] = 1
        else:
            if np.random.binomial(1, back_edge_probability):
                adj_mat[i, j] = 1

    data = np.zeros((500, number_of_nodes))
    
    for sample in range(500):
        noise = np.random.normal(scale=3, size=number_of_nodes)
        for i in range(number_of_nodes):
            data[sample, i] += np.sum(noise[:i+1] * adj_mat[:i+1, i])

    return adj_mat, data


In [ ]:
adj_mat, data = generate_random_graph(20, 0.2, 0.5)

In [ ]:
data[:10,0]

In [ ]:
results = []
for trial in range(50):
    adj_mat, data = generate_random_graph(20, 0.2, 0.5)
    adj_mat -= np.diag([1 for i in range(20)])
    for conf_lvl in [2 ** i / 100 for i in range(-4, 3)]:
        est_skel_pop = pc_pop(data, conf_lvl, 1)
        est_skel_stable = pc_stable(data, conf_lvl, 1)
#         print(f"The original graph has {np.count_nonzero(adj_mat)} edges.")
#         print(
#             f"The estimated skeleton computed by pc-pop algorithm has {np.count_nonzero(est_skel_pop)}"
#         )
#         print(
#             f"The estimated skeleton computed by pc-stable algorithm has {np.count_nonzero(est_skel_stable)}"
#         )
        recall_pop = np.sum(np.multiply(adj_mat, est_skel_pop)) / np.count_nonzero(adj_mat)
        missing_pop = np.sum(adj_mat - est_skel_pop) / np.count_nonzero(adj_mat)
        
        recall_stable = np.sum(np.multiply(adj_mat, est_skel_stable)) / np.count_nonzero(adj_mat)
        missing_stable = np.sum(adj_mat - est_skel_stable) / np.count_nonzero(adj_mat)
        
        print(f'Trial #{trial}: Confidence level: {conf_lvl}, PC-Pop Recall: {recall_pop}, PC-Pop Missing: {missing_pop}, PC-Stable Recall: {recall_stable}, PC-Stable Missing: {missing_stable}')
        results.append((conf_lvl, recall_pop, missing_pop, recall_stable, missing_stable))
